In [1]:
!wget https://data.vespa.oath.cloud/onnx_models/msmarco_multilingual.onnx 

--2021-10-28 12:53:19--  https://data.vespa.oath.cloud/onnx_models/msmarco_multilingual.onnx
Resolving data.vespa.oath.cloud (data.vespa.oath.cloud)... 2600:9000:2181:5600:e:b7f6:8c00:93a1, 2600:9000:2181:1600:e:b7f6:8c00:93a1, 2600:9000:2181:9600:e:b7f6:8c00:93a1, ...
Connecting to data.vespa.oath.cloud (data.vespa.oath.cloud)|2600:9000:2181:5600:e:b7f6:8c00:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 428017218 (408M) [binary/octet-stream]
Saving to: ‘msmarco_multilingual.onnx’

msmarco_multilingua 100%[===================>] 408,19M   860KB/s    in 7m 19s  

2021-10-28 13:00:38 (953 KB/s) - ‘msmarco_multilingual.onnx’ saved [428017218/428017218]



In [3]:
!pip3 show onnxruntime torch 

Name: onnxruntime
Version: 1.9.0
Summary: ONNX Runtime is a runtime accelerator for Machine Learning models
Home-page: https://onnxruntime.ai
Author: Microsoft Corporation
Author-email: onnxruntime@microsoft.com
License: MIT License
Location: /usr/local/anaconda3/lib/python3.8/site-packages
Requires: protobuf, numpy, flatbuffers
Required-by: 
---
Name: torch
Version: 1.7.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/anaconda3/lib/python3.8/site-packages
Requires: future, dataclasses, typing-extensions, numpy
Required-by: torchvision, torchmetrics, sentence-transformers, pytorch-lightning, optimum, reranker


In [4]:
import onnxruntime as ort
import numpy as np
session = ort.InferenceSession("msmarco_multilingual.onnx")

In [5]:
model_input = {'input_ids': np.array([[     0,     10,     41,   1294,      2,      2,     10, 149357,
             2]]), 'attention_mask': np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
session.run(input_feed=model_input,output_names=['output_0'])

[array([[1.8985014]], dtype=float32)]

In [7]:
from onnxruntime.quantization import quantize_dynamic, QuantType

In [8]:
quantize_dynamic("msmarco_multilingual.onnx", "msmarco_multilingual_int8.onnx", weight_type=QuantType.QUInt8)

In [9]:
session_int8 = ort.InferenceSession("msmarco_multilingual_int8.onnx")

In [10]:
session_int8.run(input_feed=model_input,output_names=['output_0'])

[array([[-3.6614985]], dtype=float32)]